In [1]:
from pyspark.sql import SparkSession

# SparkSession 생성
spark = SparkSession.builder \
    .appName("241211_01_SparkSQL_SQLtest") \
    .getOrCreate()


24/12/11 10:29:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
from pyspark.sql import Row


user_data = [
    Row(user_id=1, username='A', address='서울'),
    Row(user_id=2, username='B', address='대전'),
    Row(user_id=3, username='C', address='경기도'),
    Row(user_id=4, username='D', address=None),
    Row(user_id=5, username='E', address=None),
    Row(user_id=6, username='F', address='서울'),
    Row(user_id=7, username='G', address='경기도'),
    Row(user_id=8, username='H', address='대구'),
    Row(user_id=9, username='I', address='부산'),
    Row(user_id=10, username='J', address='전주'),
    Row(user_id=11, username='K', address='광주')
]


books_data = [
    Row(book_id=1, title="Book A", author_fname="John", author_lname="Doe", pages=300, released_year=2005, stock_quantity=55),
    Row(book_id=2, title="Book B", author_fname="Jane", author_lname="Smith", pages=250, released_year=2010, stock_quantity=40),
    Row(book_id=3, title="Book C", author_fname="Emily", author_lname="Jones", pages=180, released_year=2015, stock_quantity=20),
    Row(book_id=4, title="Book D", author_fname="Chris", author_lname="Brown", pages=320, released_year=2012, stock_quantity=75),
    Row(book_id=5, title="Book E", author_fname="Anna", author_lname="Davis", pages=270, released_year=2008, stock_quantity=35)
]


In [10]:
#Dataframe 생성 및 sql 테이블 생성 

user_df = spark.createDataFrame(user_data)
user_df.createOrReplaceTempView('users')

book_df = spark.createDataFrame(books_data)
book_df.createOrReplaceTempView('books')

In [11]:
spark.sql(
    '''
    select * from users;
    '''
).show()

[Stage 0:>                                                          (0 + 1) / 1]

+-------+--------+-------+
|user_id|username|address|
+-------+--------+-------+
|      1|       A|   서울|
|      2|       B|   대전|
|      3|       C| 경기도|
|      4|       D|   null|
|      5|       E|   null|
|      6|       F|   서울|
|      7|       G| 경기도|
|      8|       H|   대구|
|      9|       I|   부산|
|     10|       J|   전주|
|     11|       K|   광주|
+-------+--------+-------+



In [13]:
spark.sql(
    '''
    select username,
        IF(address IS NULL, '주소없음', address) AS address
    From users;
    '''
).show()

+--------+--------+
|username| address|
+--------+--------+
|       A|    서울|
|       B|    대전|
|       C|  경기도|
|       D|주소없음|
|       E|주소없음|
|       F|    서울|
|       G|  경기도|
|       H|    대구|
|       I|    부산|
|       J|    전주|
|       K|    광주|
+--------+--------+



In [15]:
spark.sql(
    '''
    select *
    from books
    '''
).show()

+-------+------+------------+------------+-----+-------------+--------------+
|book_id| title|author_fname|author_lname|pages|released_year|stock_quantity|
+-------+------+------------+------------+-----+-------------+--------------+
|      1|Book A|        John|         Doe|  300|         2005|            55|
|      2|Book B|        Jane|       Smith|  250|         2010|            40|
|      3|Book C|       Emily|       Jones|  180|         2015|            20|
|      4|Book D|       Chris|       Brown|  320|         2012|            75|
|      5|Book E|        Anna|       Davis|  270|         2008|            35|
+-------+------+------------+------------+-----+-------------+--------------+



In [26]:
books_1= spark.sql(
    '''
        SELECT stock_quantity, 
    	   IF(stock_quantity >= 50, '재고 많음',
    	   IF(stock_quantity >= 30, '재고 중간', '재고 없음')) AS quantity_level
        FROM books;
    '''
)
books_1.show()

+--------------+--------------+
|stock_quantity|quantity_level|
+--------------+--------------+
|            55|     재고 많음|
|            40|     재고 중간|
|            20|     재고 없음|
|            75|     재고 많음|
|            35|     재고 중간|
+--------------+--------------+



In [27]:
books_2 = spark.sql(
    '''
        SELECT stock_quantity, 
            CASE
                WHEN stock_quantity >= 50 THEN '재고 많음'
                WHEN stock_quantity >= 30 THEN '재고 중간'
                ELSE '재고 부족'
            END AS quantity_level
        FROM books;
    '''
)
books_2.show()

+--------------+--------------+
|stock_quantity|quantity_level|
+--------------+--------------+
|            55|     재고 많음|
|            40|     재고 중간|
|            20|     재고 부족|
|            75|     재고 많음|
|            35|     재고 중간|
+--------------+--------------+



In [29]:
#실행결과 비교
books_1.explain()

== Physical Plan ==
*(1) Project [stock_quantity#53L, if ((stock_quantity#53L >= 50)) 재고 많음 else if ((stock_quantity#53L >= 30)) 재고 중간 else 재고 없음 AS quantity_level#223]
+- *(1) Scan ExistingRDD[book_id#47L,title#48,author_fname#49,author_lname#50,pages#51L,released_year#52L,stock_quantity#53L]




In [30]:
books_2.explain()

== Physical Plan ==
*(1) Project [stock_quantity#53L, CASE WHEN (stock_quantity#53L >= 50) THEN 재고 많음 WHEN (stock_quantity#53L >= 30) THEN 재고 중간 ELSE 재고 부족 END AS quantity_level#235]
+- *(1) Scan ExistingRDD[book_id#47L,title#48,author_fname#49,author_lname#50,pages#51L,released_year#52L,stock_quantity#53L]




In [34]:
spark.sql( 
    '''
    select author_lname, count(*)
    from books
    group by author_lname;
'''

).show()

+------------+--------+
|author_lname|count(1)|
+------------+--------+
|       Jones|       1|
|       Davis|       1|
|       Smith|       1|
|         Doe|       1|
|       Brown|       1|
+------------+--------+



In [38]:
# books 테이블 데이터에 borrowed_by 추가
books_data_with_user = [
    Row(book_id=1, title="Book A", author_fname="John", author_lname="Doe", pages=300, released_year=2005, stock_quantity=55, borrowed_by=1),
    Row(book_id=2, title="Book B", author_fname="Jane", author_lname="Smith", pages=250, released_year=2010, stock_quantity=40, borrowed_by=2),
    Row(book_id=3, title="Book C", author_fname="Emily", author_lname="Jones", pages=180, released_year=2015, stock_quantity=20, borrowed_by=3),
    Row(book_id=4, title="Book D", author_fname="Chris", author_lname="Brown", pages=320, released_year=2012, stock_quantity=75, borrowed_by=None),
    Row(book_id=5, title="Book E", author_fname="Anna", author_lname="Davis", pages=270, released_year=2008, stock_quantity=35, borrowed_by=6)
]


books_df_with_user = spark.createDataFrame(books_data_with_user)
books_df_with_user.createOrReplaceTempView("books")

In [44]:
from pyspark.sql.functions import when, lit

# borrowed_by 컬럼 추가 및 데이터 입력
updated_books_df = book_df.withColumn(
    "borrowed_by",
    when(book_df.book_id == 1, 1)
    .when(book_df.book_id == 2, 2)
    .when(book_df.book_id == 3, 3)
    .when(book_df.book_id == 4, lit(None))
    .when(book_df.book_id == 5, 6)
    .otherwise(None)
)


In [46]:
books_sql = spark.sql(
    '''
    select * 
    from books
    '''
)

books_sql.show()

+-------+------+------------+------------+-----+-------------+--------------+-----------+
|book_id| title|author_fname|author_lname|pages|released_year|stock_quantity|borrowed_by|
+-------+------+------------+------------+-----+-------------+--------------+-----------+
|      1|Book A|        John|         Doe|  300|         2005|            55|          1|
|      2|Book B|        Jane|       Smith|  250|         2010|            40|          2|
|      3|Book C|       Emily|       Jones|  180|         2015|            20|          3|
|      4|Book D|       Chris|       Brown|  320|         2012|            75|       null|
|      5|Book E|        Anna|       Davis|  270|         2008|            35|          6|
+-------+------+------------+------------+-----+-------------+--------------+-----------+



In [49]:
#book_id = 3, stock_quantity = 50으로 바꾼다. 
#sql의 update set과 같음


updated_books_df = books_df_with_user.withColumn( 
    'stock_quantity', 
    when(books_df_with_user.book_id == 3, 50).otherwise(books_df_with_user.stock_quantity)
)

updated_books_df.show()

+-------+------+------------+------------+-----+-------------+--------------+-----------+
|book_id| title|author_fname|author_lname|pages|released_year|stock_quantity|borrowed_by|
+-------+------+------------+------------+-----+-------------+--------------+-----------+
|      1|Book A|        John|         Doe|  300|         2005|            55|          1|
|      2|Book B|        Jane|       Smith|  250|         2010|            40|          2|
|      3|Book C|       Emily|       Jones|  180|         2015|            50|          3|
|      4|Book D|       Chris|       Brown|  320|         2012|            75|       null|
|      5|Book E|        Anna|       Davis|  270|         2008|            35|          6|
+-------+------+------------+------------+-----+-------------+--------------+-----------+



In [50]:
#stock_quantity 10% 증가
books_df_with_user.withColumn(
    'stock_quantity',
    books_df_with_user.stock_quantity = books_df_with_user.stock_quantity*10%
)

SyntaxError: invalid syntax (992090041.py, line 5)

In [51]:
#뷰로 등록
updated_books_df.createOrReplaceTempView("books")
spark.sql("select * from books").show()

+-------+------+------------+------------+-----+-------------+--------------+-----------+
|book_id| title|author_fname|author_lname|pages|released_year|stock_quantity|borrowed_by|
+-------+------+------------+------------+-----+-------------+--------------+-----------+
|      1|Book A|        John|         Doe|  300|         2005|            55|          1|
|      2|Book B|        Jane|       Smith|  250|         2010|            40|          2|
|      3|Book C|       Emily|       Jones|  180|         2015|            50|          3|
|      4|Book D|       Chris|       Brown|  320|         2012|            75|       null|
|      5|Book E|        Anna|       Davis|  270|         2008|            35|          6|
+-------+------+------------+------------+-----+-------------+--------------+-----------+



## 데이터 저장

In [54]:
#write의 저장 mode: overwrite, append, ignore, error

updated_books_df.write.csv("data/output/sqltest_updated_books.csv", header = True, mode = 'overwrite')

In [55]:
user_df.write.csv("data/output/sqltest_updated_users.csv", header = True, mode = 'overwrite')